In [7]:
import randf_model as model
import pandas as pd
import numpy as np

In [ ]:
# Example usage
audio_directory = 'TAUKADIAL-24/train/'  # Replace with your actual directory path
all_audio_features = model.process_audio_directory(audio_directory)

### Load the processed data and Ground truth data into a dataframe

In [ ]:
df1 = pd.DataFrame(all_audio_features)
df1.rename(columns={0: 'tkdname', 1: 'mfccs', 2: 'chroma', 3: 'spectral_contrast', 4: 'spectral_rolloff', 5: 'zero_crossing_rate', 6: 'rmse'}, inplace=True)

df2 = pd.read_csv('TAUKADIAL-24/train/groundtruth.csv')

df = pd.merge(df1, df2, on='tkdname')
df.to_csv('TAUKADIAL-24.csv')

In [ ]:
df.head()

In [ ]:
data = df.copy()
data.drop(columns=['tkdname'], inplace=True)
data['mfccs'] = data['mfccs'].apply(lambda x: np.mean(x))
data['chroma'] = data['chroma'].apply(lambda x: np.mean(x))
data['spectral_contrast'] = data['spectral_contrast'].apply(lambda x: np.mean(x))


data['sex'] = data['sex'].map({'M': 1, 'F': 0})

data.drop_duplicates(inplace=True)
data.groupby('dx').describe()

### Train the model

In [ ]:
# Define features and target
features = ['mfccs', 'chroma', 'spectral_contrast', 'spectral_rolloff', 'zero_crossing_rate', 'rmse', 'age', 'sex', 'mmse']
target = 'dx'

# Train the model
trained_model, test_features, test_target = model.train_model(data, features, target)

### Evaluate the model

In [ ]:
# Test the model
ypred, ytest, report = model.test_model(trained_model, test_features, test_target)

In [ ]:
target_labels = ['MCI','NC']
model.plot_confusion_matrix(ytest, ypred,target_labels)

In [ ]:
import matplotlib.pyplot as plt

features = ['mfccs', 'chroma', 'spectral_contrast', 'spectral_rolloff', 'zero_crossing_rate', 'rmse', 'age', 'sex', 'mmse']
preprocessor = trained_model.named_steps['preprocessor']
print(preprocessor.transformers_)


importances = trained_model.named_steps['classifier'].feature_importances_


indices = np.argsort(importances)[::-1]
print(indices)
plt.figure(figsize=(10, 6))
plt.bar(range(len(features)), importances[indices], align='center')
plt.xticks(range(len(features)), [features[i] for i in indices], rotation=45)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.show()

In [ ]:
report